In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

image_path = 'C:/Users/tabm9/OneDrive - Caissa Analytica/Documents/DS_Portfolio/Projects/Images/RLCS'

# Test with only 2 teams

In [16]:
def run_games(Pa=1/4, K=1, N=10000, thres=0.10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = [0]
    Rb = [0]
    pa = [1/2]
    finished_burnout = False
    n_to_convergence = N + 1
    for i in range(N):
        # Update rating
        Ra.append(Ra[-1] + K * (Sa[i] - pa[-1]))
        Rb.append(Rb[-1] + K * (-Sa[i] + pa[-1]))

        # Update Approx Probabilities
        pa.append(1 / (1 + 10 ** ((Rb[-1] - Ra[-1]) / 400)))

        # Check if burnout is finished
        if not finished_burnout:
            if abs(Pa - pa[-1]) <= Pa * thres:
                finished_burnout = True
                n_to_convergence = i + 1
                if stop_burnout: break

    return pa, n_to_convergence
        

In [21]:
Pa = 1/4
N = 10000
K = 10
pa, n_to_convergence = run_games(Pa=Pa, N=N, K=K)

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(N + 1)), y=[Pa] * (N + 1), line=dict(color='black'), opacity=0.5, name='Real Probability'))
fig.add_trace(go.Scatter(x=list(range(N + 1)), y=pa, name='Approximated Probability'))

fig.show()


In [154]:
def logbase(base, x):
    return np.log(x) / np.log(base)

def K_generator(K_interval=[0,100], K_default=80):
    eps = (K_interval[1] - K_interval[0]) * np.e ** (-2 * np.e)
    K_d = (K_default - K_interval[0]) / (K_interval[1] - K_interval[0])
    b = logbase(1/2, 1/2 - np.log((1 + np.e ** (-2 * np.e) - K_d) / (K_d + np.e ** (-2 * np.e))) / (4 * np.e))

    def K(x, eps=eps, b=b, K_interval=K_interval):
        return (K_interval[1] - K_interval[0]) * ((1 + 2 * np.e ** (-2 * np.e)) / (1 + np.e ** (-4 * np.e * (x ** b - 1/2))) - np.e ** (-2 * np.e)) + K_interval[0]

    return K



def run_inertia_games(Pa=Pa, innitial_K=1, N=10000, thres=0.10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = [0]
    Rb = [0]
    pa = [1/2]
    finished_burnout = False
    n_to_convergence = N + 1
    K = K_generator()
    K_list = [innitial_K]
    for i in range(N):
        # Update rating
        # min_dif = min(pa[-1], 1 - pa[-1])
        # inertia_update = (abs(Sa[i] - pa[-1]) - min_dif)
        inertia_update = abs(Sa[i] - pa[-1])

        K_i = K(abs(pa[-1] - sum(Sa[:i]) / (i + 1)))
        Ra.append(Ra[-1] + K_i * (Sa[i] - pa[-1]))
        Rb.append(Rb[-1] + K_i * (-Sa[i] + pa[-1]))

        K_list.append(K_i)

        # Update Approx Probabilities
        pa.append(1 / (1 + 10 ** ((Rb[-1] - Ra[-1]) / 400)))

        # Check if burnout is finished
        if not finished_burnout:
            if abs(Pa - pa[-1]) <= Pa * thres:
                finished_burnout = True
                n_to_convergence = i + 1
                if stop_burnout: break

    return pa, n_to_convergence, K_list

In [159]:
Pa = 1/4
N = 10000
K = 1
pa, n_to_convergence = run_games(Pa=Pa, N=N, K=K)
pa_inertia, n_to_convergence_inertia, K_list = run_inertia_games(Pa=Pa, N=N, innitial_K=K)

print(n_to_convergence, n_to_convergence_inertia)

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(N + 1)), y=[Pa] * (N + 1), line=dict(color='black'), opacity=0.5, name='Real Probability'))
fig.add_trace(go.Scatter(x=list(range(N + 1)), y=pa, name='Approximated Probability'))
fig.add_trace(go.Scatter(x=list(range(N + 1)), y=pa_inertia, name='Approximated Probability with inertia'))

fig.show()

774 4
